In [ ]:
from flask import Flask, request, render_template_string, send_file, redirect, flash, url_for, session
from werkzeug.utils import secure_filename
import numpy as np
import pandas as pd
import pickle
import faiss
from datetime import datetime
from sklearn.cluster import KMeans
from io import BytesIO
import os

app = Flask(__name__)
app.config['upload_folder'] = '/root/server/input'
app.secret_key = 'fdsgfshhgfshgjydjy54664ghfdghd'

html = '''
<!DOCTYPE html>
<html>
<head>
   <title>Введите данные</title>
   <style>
       body {
           font-family: Arial, sans-serif;
           background-color: #f4f4f9;
           padding: 50px;
       }
       form {
           background: white;
           padding: 20px;
           border-radius: 8px;
           box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
       }
       h2 {
           text-align: center;
       }
       .input-group {
           margin-bottom: 10px;
       }
       input[type=text], input[type=number], select, input[type=file] {
           width: calc(100% - 22px);
           padding: 10px;
           border: 1px solid #ddd;
           border-radius: 4px;
       }
       input[type=submit] {
           background-color: #0056b3;
           color: white;
           border: none;
           padding: 10px 20px;
           border-radius: 4px;
           cursor: pointer;
       }
       input[type=submit]:hover {
           background-color: #004494;
       }
       .flashes {
           list-style-type: none;
           padding: 0;
           color: red;
       }
       .information {
           background: #fff;
           padding: 10px;
           margin-bottom: 20px;
           border-radius: 5px;
           box-shadow: 0 2px 5px rgba(0,0,0,0.1);
           font-size: 14px;
           line-height: 20px;
       }

   </style>
</head>
<body>
  <div class="information">
    <h4>Инструкция:</h4>
    <pre>
      ЗАГОЛОВОК
    </pre>
  </div>
  <h2>Look Alike</h2>
  <!-- Вывод сообщений Flask -->
  {% with messages = get_flashed_messages() %}
    {% if messages %}
      <ul class="flashes">
      {% for message in messages %}
        <li>{{ message }}</li>
      {% endfor %}
      </ul>
    {% endif %}
  {% endwith %}
  <!-- Форма для загрузки файла -->
  <form method="post" action="{{ url_for('upload_file')}}" enctype="multipart/form-data">
      <div class="input-group">
          <label for="file-upload">Загрузите файл:</label>
          <input type="file" id="file-upload" name="file">
          <input type="submit" value="Загрузить файл">
      </div>
  </form>
 <!-- Существующая форма для ввода данных -->
 <form method="get" action="/download">
<div class="input-group">
    Префиксы: <input type="text" id="prefixes_lc" name="prefixes_lc" readonly>
    <select id="prefix-select-lc" multiple size="6">
        <option value="i_">i_</option>
        <option value="b_">b_</option>
        <option value="l_">l_</option>
        <option value="s_">s_</option>
        <option value="f_">f_</option>
        <option value="p_">p_</option>
    </select>
</div>
<div class="input-group">
    Префиксы: <input type="text" id="prefixes_ie" name="prefixes_ie" readonly>
    <select id="prefix-select-ie" multiple size="4">
        <option value="i_">i_</option>
        <option value="b_">b_</option>
        <option value="l_">l_</option>
        <option value="p_">p_</option>
    </select>
</div>

     <div class="input-group">
         Количество: <input type="number" name="inn_out_count_lc">
     </div>
     
     <div class="input-group">
         Количество : <input type="number" name="inn_out_count_ie">
     </div>   
      
      <input type="submit" value="Отправить">
  </form>
   <!-- Форма для скачивания Excel файла -->
   <form method="get" action="/download"style="display: none;">
       <input type="submit" value="Download Excel">
   </form>
    <!-- JavaScript для обработки выбора префиксов -->
<script>
document.addEventListener('DOMContentLoaded', function() {
    var selectLc = document.getElementById('prefix-select-lc');
    var inputLc = document.getElementById('prefixes_lc');
    var selectIe = document.getElementById('prefix-select-ie');
    var inputIe = document.getElementById('prefixes_ie');


    // Обработчик 
    selectLc.addEventListener('change', function() {
        var selectedOptions = Array.from(selectLc.selectedOptions).map(option => option.value).join(',');
        inputLc.value = selectedOptions;
    });


    // Обработчик
    selectIe.addEventListener('change', function() {
        var selectedOptions = Array.from(selectIe.selectedOptions).map(option => option.value).join(',');
        inputIe.value = selectedOptions;
    });
});
</script>

</body>
</html>
'''


@app.route('/upload', methods=['POST'])
def upload_file():
    file = request.files.get('file')
    if not file:
        flash('Файл не найден')
        return redirect(request.url)
    if file.filename == '':
        flash('Файл не выбран')
        return redirect(request.url)
    if not file.filename.lower().endswith('.xlsx'):
        flash('Недопустимый тип файла')
        return redirect(request.url)

    # Чтение файла в памяти для проверки
    file_stream = file.read()
    try:
        df = pd.read_excel(BytesIO(file_stream), header=None, dtype ={0:str})
        inns = df[0].astype(str).tolist()
        # Проверка условий на длину ИНН
        if all(len(inn) == 10 or len(inn) == 12 for inn in inns):
            timestamp = datetime.now().strftime("%Y%m%d%H%M")
            filename = f"input_{timestamp}.xlsx"
            filepath = os.path.join(app.config['upload_folder'], filename)

            # Сохраняем файл
            with open(filepath, 'wb') as f:
                f.write(file_stream)
            session['filename'] = filename

            flash(f'Файл {filename} успешно загружен')
        else:
            flash(
                'Файл содержит неверную информацию')
    except Exception as e:
        flash(f'Ошибка при обработке файла: {str(e)}')

    return redirect(url_for('form'))



# Функция для обработки
def process_data(..... FLag_=0):

        # Получаем данные из формы
    col1 = prefixes_
    col2 = inn_out_count_
    col3 = inn_

    # Словарь введенных значений
    test_d = {
        'name1': col1,
        'name2': col2,
        'name3': col3
    }

    df_faiss = pd.read_pickle(f"/root/server/data/{base_}")
    Flag = FLag_
    inn_req = inn_req_


    
    return df_out


@app.route('/', methods=['GET'])
def form():
    return render_template_string(html)


@app.route('/download', methods=['GET','POST'])
def run():
    Flag = 0 
    filename = session.get('filename')
    if filename:
        filepath = os.path.join('/root/server/input',filename)
        df_inn = pd.read_excel(filepath, header=None, dtype ={0:str}, sheet_name = None)
    else:
        print("Файл не найден")

    # с первой страницы файла
    df_first_page = df_inn[list(df_inn.keys())[0]]
    inn_lc = list(df_first_page[df_first_page[0].str.len()==10][0])
    inn_ie = list(df_first_page[df_first_page[0].str.len() == 12][0])

    # со второй страницы файла
    if len(df_inn) > 1:
        df_second_page = df_inn[list(df_inn.keys())[1]]
        inn_lc_req = list(df_second_page[df_second_page[0].str.len() == 10][0])
        inn_ie_req = list(df_second_page[df_second_page[0].str.len() == 12][0])
        Flag = 1
    else:
        inn_lc_req = []
        inn_ie_req =[]
        Flag = 0

    # Получаем данные из формы
    prefixes_lc = request.args.get('prefixes_lc', '').split(',')
    prefixes_ie = request.args.get('prefixes_ie', '').split(',')

    if request.args.get('inn_out_count_lc') == '':
        inn_out_count_lc = 0
    elif Flag == 1 and len(inn_lc_req) > 0:
        inn_out_count_lc = len(inn_lc_req)
    else:
        inn_out_count_lc = int(request.args.get('inn_out_count_lc', 0))

    if request.args.get('inn_out_count_ie') == '':
        inn_out_count_ie = 0
    elif Flag == 1 and len(inn_ie_req) > 0:
        inn_out_count_ie = len(inn_ie_req)
    else:
        inn_out_count_ie = int(request.args.get('inn_out_count_ie', 0))

    base_lc = "df_ls_faiss.pkl"
    base_ie = "df_ie_faiss.pkl"

    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        # Обрабатываем юридические лица
        if len(inn_lc) > 0:
            df_out_lc = process_data(..... Flag)
            df_out_lc.to_excel(writer, index=False, sheet_name='res_lc')
        else:
            # Создаем пустой DataFrame и записываем в лист, если список пуст
            pd.DataFrame().to_excel(writer, index=False, sheet_name='res_lc')

        # Обрабатываем индивидуальных предпринимателей
        if len(inn_ie) > 0:
            df_out_ie = process_data(.....Flag)
            df_out_ie.to_excel(writer, index=False, sheet_name='res_ie')
        else:
            # Создаем пустой DataFrame и записываем в лист, если список пуст
            pd.DataFrame().to_excel(writer, index=False, sheet_name='res_ie')


    output.seek(0)
    file_path = os.path.join('/root/server/output','output_'+ filename.split('.')[0].split('_')[1]+'.xlsx')
    with open(file_path, 'wb') as f:
        f.write(output.read())
    output.seek(0)

    # Отправляем файл пользователю
    return send_file(output, as_attachment=True, download_name="result.xlsx")
